In [12]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=100,
    return_messages=True,
)

examples = [
    {"input": "엘리멘탈", "output": "🔥💧👨‍👨‍👦"},
    {"input": "겨울왕국", "output": "🥶👑⛄️"},
    {"input": "오페라의 유령", "output": "🎭👻🎤"},
    {"input": "라따뚜이", "output": "🐭👨🏻‍🍳🍲"},
    {"input": "탑건", "output": "🛩️👨‍✈️🔥"},
    {"input": "대부", "output": "👨‍👨‍👦🔫🍝"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "영화 제목: {input}"),
        ("ai", "영화 제목: {input}, 이모지 요약: {output}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 영화 제목을 입력받으면, 그 영화의 줄거리를 상징하는 이모지 3개로 요약해주십시오. 가능한 한 정확하고 상징적인 이모지를 선택해야 합니다.",
        ),
        MessagesPlaceholder(variable_name="history"),
        example_prompt,
        ("human", "영화 제목: {input}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(input):
    result = chain.invoke({"input": input})
    memory.save_context(
        {"input": input},
        {"output": result.content},
    )
    print(result.content)


invoke_chain("아메리칸 쉐프")
invoke_chain("주토피아")
invoke_chain("명량")


영화 제목: 아메리칸 쉐프, 이모지 요약: 🇺🇸👨🏻‍🍳🍔
영화 제목: 주토피아, 이모지 요약: 🐭🌆👮‍♂️
영화 제목: 명량, 이모지 요약: ⛵🌊💥
